In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("Financial"))))

In [2]:
from Collection.Financial import FMP as fp

# Collect

In [2]:
fp.information()


        함수에 대한 설명은 아래와 같습니다. 

        라이브러리 내 주요 클래스는 fmp_extact입니다. 

        get_jsonparsed_data()는 데이터를 파싱하는 함수입니다. 

        extractor()은 데이터를 json형태로 가지고 오는 함수입니다. 

        url_generator()은 FMP 사이트에 접속하여 데이터를 분리하는 함수입니다. 

        ending_period_extact()는 날짜를 표준화하는 함수입니다. 

        report_type_extract()는 들어오는 값에 따라 연간인지 분기인지 구분하는 함수입니다. 

        GetExcel()은 추출한 데이터를 저장하는 함수입니다. 

        cleanse()는 데이터를 가공하는 함수입니다. 

        get_symbols()는 데이터를 사이트로부터 가져오는 함수입니다. 

        make_clean()은 위의 함수들을 순차적으로 실행하여 데이터를 추출 후 저장하는 함수입니다.
        


In [3]:
fps = fp.fmp_extract()

# Transform

In [3]:
fps = fp.fmp_extract()

In [4]:
filepath = os.listdir("og/fmp/")

In [5]:
for idx in filepath:
    fps.cleanse(f"og/fmp/{idx}", f"finished/{idx}")

OSError: Cannot save file into a non-existent directory: 'finished_finished'